In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# root_path = "/content/drive/MyDrive/GRSL/RS_INDEX/"

In [ ]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.0 MB/s eta 0:00:00


In [ ]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 10.7 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_addons as tfa
from PIL import Image
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix
import shap

In [ ]:
# load the whole dataset, for data info
all_ds   = tfds.load("eurosat/all", with_info=True)
# load training, testing & validation sets, splitting by 60%, 20% and 20% respectively
train_ds = tfds.load("eurosat/all", split="train[:95%]")
# test_ds  = tfds.load("eurosat/all", split="train[90%:]")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/eurosat/all/2.0.0.incompleteOIABKR/eurosat-train.tfrecord*...:   0%|      …

Dataset eurosat downloaded and prepared to /root/tensorflow_datasets/eurosat/all/2.0.0. Subsequent calls will reuse this data.


In [ ]:
# the class names
class_names = all_ds[1].features["label"].names
# total number of classes (10)
num_classes = len(class_names)
num_examples = all_ds[1].splits["train"].num_examples

In [ ]:
# number of testing steps
n_training_steps = int(all_ds[1].splits["train"].num_examples * 0.95)

# get all testing images as NumPy array
images = np.array([ d["sentinel2"] for d in train_ds.take(n_training_steps) ])
print("images.shape:", images.shape)

# get all testing labels as NumPy array
labels = np.array([ d["label"] for d in train_ds.take(n_training_steps) ])
print("labels.shape:", labels.shape)

images.shape: (25650, 64, 64, 13)
labels.shape: (25650,)


In [ ]:
ind_train = []

for i in range(0,images.shape[0]):
  image1 = images[i]
  red = image1[:,:,3]
  # red = np.uint8((red_raw /(red_raw.max()))*255)
  # print(red.min(), red.max())

  nir = image1[:,:,7]
  # nir = np.uint8(((nir_raw/(nir_raw.max())))*255)
  #print(nir.min(), nir.max())

  swir1 = image1[:,:,12]
  # swir1 = np.uint8(((swir1_raw/swir1_raw.max()))*255)
  #print(swir1.min(), swir1.max())

  green = image1[:,:,2]
  # green = np.uint8(((green_raw/(green_raw.max())))*255)
  #print(green.min(), green.max())

  bleu = image1[:,:,1]
  # bleu = np.uint8(((bleu_raw/(bleu_raw.max())))*255)
  #print(bleu.min(), bleu.max())
  #print("")

  ndvi = (nir - red)/(nir + red)
  ndvi = (ndvi - ndvi.min())/(ndvi.max() - ndvi.min())
  # #print(ndvi.min(), ndvi.max())

  ndwi = (green - nir)/(green+nir)
  ndwi = (ndwi - ndwi.min())/(ndwi.max() - ndwi.min())
  # #print(ndwi.min(), ndwi.max())

  ndsi = (swir1 - nir )/(swir1 + nir )
  ndsi = (ndsi - ndsi.min())/(ndsi.max() - ndsi.min())
  # #print(ndbi.min(), ndbi.max())

  st_indices2 = np.stack([ndsi, ndvi, ndwi], axis=2)
  ind_train.append(st_indices2)
  # print(st_indices2.min())
  # print(st_indices2.max())


INDICIES = np.array(ind_train)
print('INDICIES shape:', INDICIES.shape)
print('INDICIES shape:', INDICIES.dtype)

In [ ]:
img = Image.fromarray(np.uint8(INDICIES[0]*255), mode = "RGB")
plt.title(labels[0])
plt.imshow(img)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(INDICIES, labels, test_size=0.20, random_state=42, shuffle = True)

In [ ]:
# X_train = X_train.astype("float32") / 255
# X_test = X_test.astype("float32") / 255

print(X_train.shape)
print(X_train.dtype)

print(y_train.shape)
print(y_train.dtype)

print(X_test.shape)
print(X_test.dtype)

print(y_test.shape)
print(y_test.dtype)

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.compat.v1.disable_v2_behavior()

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(64,
                                  64,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
filters = 16
model = Sequential([
  data_augmentation,

  layers.Conv2D(filters, 3, padding='same', kernel_initializer='he_normal', activation=tfa.activations.gelu),
  # layers.BatchNormalization(),
  layers.Conv2D(filters, 3, padding='same', kernel_initializer='he_normal', activation=tfa.activations.gelu),
  # layers.BatchNormalization(),
  layers.MaxPooling2D(),

  layers.Conv2D(2*filters, 3, padding='same', kernel_initializer='he_normal', activation=tfa.activations.gelu),
  # layers.BatchNormalization(),
  layers.Conv2D(2*filters, 3, padding='same', kernel_initializer='he_normal', activation=tfa.activations.gelu),
  # layers.BatchNormalization(),
  layers.MaxPooling2D(),

  layers.Conv2D(4*filters, 3, padding='same', kernel_initializer='he_normal', activation=tfa.activations.gelu),
  # layers.BatchNormalization(),
  layers.Conv2D(4*filters, 3, padding='same', kernel_initializer='he_normal', activation=tfa.activations.gelu),
  # layers.BatchNormalization(),
  layers.MaxPooling2D(),

  layers.Conv2D(8*filters, 3, padding='same', kernel_initializer='he_normal', activation=tfa.activations.gelu),
  # layers.BatchNormalization(),
  layers.Conv2D(8*filters, 3, padding='same', kernel_initializer='he_normal', activation=tfa.activations.gelu),
  # layers.BatchNormalization(),
  layers.MaxPooling2D(),

  layers.Conv2D(16*filters, 3, padding='same', kernel_initializer='he_normal', activation=tfa.activations.gelu),
  # layers.BatchNormalization(),
  layers.Conv2D(16*filters, 3, padding='same', kernel_initializer='he_normal', activation=tfa.activations.gelu),
  # layers.BatchNormalization(),
  layers.MaxPooling2D(),

  layers.Flatten(),

  layers.Dense(32*filters, activation=tfa.activations.gelu),
  layers.Dropout(0.1),
  layers.Dense(16*filters, activation=tfa.activations.gelu),
  layers.Dropout(0.2),
  layers.Dense(8*filters, activation=tfa.activations.gelu),
  layers.Dropout(0.3),
  layers.Dense(num_classes, activation = "softmax")
])

model.compile(optimizer=tfa.optimizers.LAMB(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

In [ ]:
# fiting the model
epochs = 100

# Callback options for preventing overfitting (earlystopping)
callbacksOptions = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=15, min_lr=0.0001, verbose=1),

    # save best only result
    keras.callbacks.ModelCheckpoint('checkpoint.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
history = model.fit(
  X_train, y_train,
  validation_split=0.10,
  epochs=epochs,
  callbacks=callbacksOptions
)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


In [ ]:
# feed the images to get predictions
predictions = model.predict(X_test)
# perform argmax to get class index
predictions = np.argmax(predictions, axis=1)
print("predictions.shape:", predictions.shape)

In [ ]:
cf_matrix = confusion_matrix(y_test, predictions)

plt.figure(figsize=(10, 10))
sns.heatmap(cf_matrix, annot=False, fmt='.0f',  xticklabels=[f"{c}" for c in class_names],
            yticklabels=[f"{c}" for c in class_names],
            # cmap="Blues"
            cmap="Blues")

plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.ylabel('Actual', fontsize=20)
plt.xlabel('Predicted', fontsize=20)

# plt.savefig(root_path+'conf_rs_index.eps', bbox_inches='tight', format = 'eps')
# plt.savefig(root_path+'conf_rs_index.svg', bbox_inches='tight', format = 'svg')

In [ ]:
import sklearn
print("\nClassification Report :")
print(sklearn.metrics.classification_report(y_test, predictions, target_names=class_names, digits = 4))

In [ ]:
# select backgroud for shap
background = X_train[np.random.choice(X_train.shape[0], 800, replace=False)]# DeepExplainer to explain predictions of the model
explainer = shap.DeepExplainer(model, background)# compute shap values

In [ ]:
for i in range(100):
  print(i)
  num = i
  n_num= num+1

  # shap_values = explainer.explainer.shap_values(X_test[num:n_num], check_additivity=False)
  shap_values = explainer.shap_values(X_test[num:n_num], check_additivity=False)
  #shap_values = explainer.shap_values(x_test_each_class)

  predictions = model.predict(X_test[num:n_num])
  predicted_class = np.argmax(predictions, axis=1)
  print("The predicted class is ", class_names[int(predicted_class)], predicted_class)
  print("The actual class is ", class_names[int(y_test[num:n_num])], y_test[num:n_num])

  plt.figure()
  # plot the feature attributions
  fig = shap.image_plot(shap_values, X_test[num:n_num], width = 25, labels= class_names, show=False)

  # # displaying the title
  # plt.savefig(root_path+str(class_names[int(predicted_class)])+'_'+str(class_names[int(y_test[num:n_num])])+'1.eps', bbox_inches='tight', format = 'eps')
  # plt.savefig(root_path+str(class_names[int(predicted_class)])+'_'+str(class_names[int(y_test[num:n_num])])+'1.svg', bbox_inches='tight', format = 'svg')

  plt.show()